# Trabalho 1 - MC853B

**Alunos:** 

Lucca Miranda Nunes - XXXX

Luiz Felipe Corradini Rego Costa - 230613

Pablo Henrique Almeida Mendes - XXXX




**Doença de notificação compulsória escolhida:** Esquistossomose

## 1) Upload e investigação dos dados

### 1.1) Análise Exploratória dos dados

Os dados escolhidos para treinamento foram os da doença esquistossomose dos anos de 2021, 2022 e 2023. Os anos de 2024 em diante não estavam disponibilizados na base do SINAN. Todos os dados estão no diretório source/csv.

In [78]:
import pandas as pd

esq_20 = pd.read_csv('source/csv/ESQUBR20.csv')
esq_21 = pd.read_csv('source/csv/ESQUBR21.csv')
esq_22 = pd.read_csv('source/csv/ESQUBR22.csv')
esq_23 = pd.read_csv('source/csv/ESQUBR23.csv')

In [79]:
print(esq_20.shape)
print(esq_21.shape)
print(esq_22.shape)
print(esq_23.shape)

(1873, 54)
(2395, 54)
(2640, 54)
(3059, 54)


Com os três anos selecionados, obtemos, em primeiro momento, um total de 8094 instâncias. 

Precisamos investigar a quantidade de atributos-alvo em cada ano, a fim de garantir balanceamento nas classes dentro dos datasets de treino e de teste:

Valores correspondentes a cada desfecho possível do paciente:


1 - Cura

2 - Não Cura

3 - Óbito por 
esquistossomose 

4 -  Óbito por outras 
causas 

9 - Ignorado


Como iremos usar os dados dos três anos, faz mais sentido concatená-los em um único DataFrame, a fim de realizar a análise em um único comando

In [80]:
esq_total = pd.concat([esq_20, esq_21, esq_22, esq_23])
esq_total['EVOLUCAO'].value_counts()

1.0    4901
9.0    1121
3.0     242
2.0     225
4.0     113
Name: EVOLUCAO, dtype: int64

In [81]:
print(f"Porcentagem de mortes por esquistossomose: {len(esq_total[esq_total['EVOLUCAO'] == 3]) / len(esq_total) * 100 :.2f}%")

Porcentagem de mortes por esquistossomose: 2.43%


### 1.2) Divisão do conjunto de treino e teste

O primeiro passo vai ser separar um conjunto de teste nos dados, já que esse conjunto não deve ser pré-processado como os de treinamento. Para garantir que 

In [ ]:
# Avaliar a janela de tempo entre o primeiro atendimento e a morte
# Avaliar como vai ser a divisão dos dados, e avaliar também como garantir 10% de representatividade de cada classe nos conjuntos de treino e teste

## 2) Pré-processamento dos dados

Podemos ver que, pelo número elevado de colunas, será necessário realizar uma análise quanto às colunas necessárias, para posteriormente avaliar se o total de instâncias apresentadas nos três anos serão suficientes para treinar o modelo.

### 2.1) Removendo Colunas desnecessárias

O primeiro passo para pré-processar será **remover colunas desnecessárias do dataset**. 

Depois de uma primeira análise, a equipe selecionou 11 colunas que julgamos como mais relevantes para o treinamento do modelo:


*colocar info das colunas*


In [83]:
esq_total.columns

Index(['Unnamed: 0', 'TP_NOT', 'ID_AGRAVO', 'DT_NOTIFIC', 'SEM_NOT', 'NU_ANO',
       'SG_UF_NOT', 'ID_MUNICIP', 'ID_REGIONA', 'ID_UNIDADE', 'DT_SIN_PRI',
       'SEM_PRI', 'ANO_NASC', 'NU_IDADE_N', 'CS_SEXO', 'CS_GESTANT', 'CS_RACA',
       'CS_ESCOL_N', 'SG_UF', 'ID_MN_RESI', 'ID_RG_RESI', 'ID_PAIS',
       'DT_INVEST', 'ID_OCUPA_N', 'DT_COPRO', 'AN_QUANT', 'OUTROS', 'TRATAM',
       'TRATANAO', 'STCURA1', 'STCURA2', 'STCURA3', 'FORMA', 'TPAUTOCTO',
       'COUFINF', 'COPAISINF', 'COMUNINF', 'NOPROPIN', 'NOCOLINF',
       'DOENCA_TRA', 'EVOLUCAO', 'DT_ENCERRA', 'DT_DIGITA', 'DT_TRANSUS',
       'DT_TRANSDM', 'DT_TRANSSM', 'DT_TRANSRS', 'DT_TRANSSE', 'DT_OBITO',
       'DS_FORMA', 'AN_QUALI', 'DTTRAT', 'DT_RESU3', 'OUTRO_EX'],
      dtype='object')

In [84]:
# Lista das colunas conforme dicionário SINAN
colunas_selecionadas = [
    'CS_GESTANT',                     
    'AN_QUANT',
    'AN_QUALI',            
    'TRATAM', 
    'TRATANAO',    
    'FORMA',      
    'CS_ESCOL_N', 
    'CS_SEXO',  
    'ANO_NASC',
    'COUFINF',
    'EVOLUCAO'
]

# Seleção das colunas
esq_filtrado = esq_total[colunas_selecionadas]

esq_filtrado.head()


,CS_GESTANT,AN_QUANT,AN_QUALI,TRATAM,TRATANAO,FORMA,CS_ESCOL_N,CS_SEXO,ANO_NASC,COUFINF,EVOLUCAO
0,6,0.0,3.0,3.0,3.0,4.0,9.0,M,1969.0,31.0,2.0
1,6,1.0,1.0,1.0,NaN,NaN,NaN,M,1984.0,26.0,1.0
2,5,0.0,NaN,1.0,NaN,NaN,1.0,F,1975.0,35.0,1.0
3,5,0.0,1.0,3.0,9.0,NaN,1.0,F,1977.0,29.0,NaN
4,5,0.0,1.0,1.0,NaN,1.0,9.0,F,2004.0,27.0,1.0


Renomeando as colunas selecionadas para garantir melhor entendimento:

In [85]:
novos_nomes_doenças = {
    'CS_GESTANT': 'Gestante',
    'AN_QUANT': 'Quantidade de ovos encontrados',
    'AN_QUALI': 'Resultado do exame de ovos da doença',
    'TRATAM': 'Tratamento realizado',
    'TRATANAO': 'Motivo tratamento não realizado',
    'FORMA': 'Forma clínica',
    'CS_ESCOL_N': 'Escolaridade',
    'CS_SEXO': 'Sexo',
    'ANO_NASC': 'Ano de nascimento',
    'COUFINF': 'Unidade Federativa',
    'EVOLUCAO': 'Resultado'

}

esq_filtrado = esq_filtrado.rename(columns=novos_nomes_doenças)

print(esq_filtrado.shape)
esq_filtrado.head()

(9967, 11)


,Gestante,Quantidade de ovos encontrados,Resultado do exame de ovos da doença,Tratamento realizado,Motivo tratamento não realizado,Forma clínica,Escolaridade,Sexo,Ano de nascimento,Unidade Federativa,Resultado
0,6,0.0,3.0,3.0,3.0,4.0,9.0,M,1969.0,31.0,2.0
1,6,1.0,1.0,1.0,NaN,NaN,NaN,M,1984.0,26.0,1.0
2,5,0.0,NaN,1.0,NaN,NaN,1.0,F,1975.0,35.0,1.0
3,5,0.0,1.0,3.0,9.0,NaN,1.0,F,1977.0,29.0,NaN
4,5,0.0,1.0,1.0,NaN,1.0,9.0,F,2004.0,27.0,1.0


### 2.2) Selecionando os dados a serem usados

Como o problema que iremos prever é de natureza binária (óbito ou não-óbito), precisamos remover todas as instâncias que a resposta foi "Ignorado" ou "Não-Cura". Se o caso foi ignorado, não podemos inserí-lo em nenhuma das duas classes, e a classificação de "Não-Cura" também não torna possível avaliar se o paciente veio a óbito ou não. Além disso, avaliamos que fugia do escopo do projeto considerar as mortes por outros fatores, então também retiramos essa classe da consideração.

In [96]:
esq_valid = esq_filtrado[(esq_filtrado['Resultado'] == 1) | (esq_filtrado['Resultado'] == 3)]
esq_valid['Resultado'].value_counts()

1.0    4901
3.0     242
Name: Resultado, dtype: int64

In [98]:
print(f"Porcentagem de mortes por esquistossomose: {len(esq_valid[esq_valid['Resultado'] == 3]) / len(esq_valid) * 100 :.2f}%")

Porcentagem de mortes por esquistossomose: 4.71%


### 2.3) Tratamento de atributos categóricos

Primeiramente, vamos visualizar nossos dados a fim de transformar os atributos categóricos em numéricos:

In [100]:
esq_valid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5143 entries, 1 to 3058
Data columns (total 11 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Gestante                              5143 non-null   int64  
 1   Quantidade de ovos encontrados        5143 non-null   float64
 2   Resultado do exame de ovos da doença  4053 non-null   float64
 3   Tratamento realizado                  5143 non-null   float64
 4   Motivo tratamento não realizado       353 non-null    float64
 5   Forma clínica                         3709 non-null   float64
 6   Escolaridade                          4551 non-null   float64
 7   Sexo                                  5143 non-null   object 
 8   Ano de nascimento                     5112 non-null   float64
 9   Unidade Federativa                    4102 non-null   float64
 10  Resultado                             5143 non-null   float64
dtypes: float64(9), in

Podemos notar que, apesar de que o tipo de muitos dos atributos esteja definido como 'float', de acordo com o dicionário da doença, cada número representa uma característica distinta. Por isso, a ideia será realizar o One-Hot encoding para separar tais atributos categóricos.

In [103]:
esq_valid['Gestante'].value_counts()

6    3573
5    1422
9     111
2      14
4      12
1       8
3       3
Name: Gestante, dtype: int64